<a href="https://colab.research.google.com/github/VLADISLAV008/DSL/blob/main/Job3/Job%233.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementation of FIRST and FOLLOW functions

We are given a context free grammar $G(\Sigma, N, S \in N, P)$

We assume that in our grammar there is no useless nonterminal symbols and left recurtion!

To start with, we will use the following context free grammar structure:
```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token: (class, value)
class: int
value: str
var: str                 # name of the non-terminal symbol
definition: list(rule)
rule: list(var | token)  # right part of the rule
```

Let's implement a method that returns all nullable (vanishing) symbols.

A nonterminal symbol $X$ is called nullable, if $X \twoheadrightarrow e$.

We will expand the set of nullable symbols as it grows.

$V_0 = \emptyset$

$V_1 = \{X \in N | \space (X \to e) \in P\}$
$V_2 = \{X \in N | \space \exists k \in \mathbb{N} \space \exists B_1, B_2, ..., B_k \in V_1, (X \to B_1 B_2 ... B_k) \in P\}$

...

$V_{n+1} = \{X \in N | \space \exists k \in \mathbb{N} \space \exists B_1, B_2, ..., B_k \in V_n, (X \to B_1 B_2 ... B_k) \in P\}$

$V_n = V_{n+1} \supset N$ - the desired set of nullable symbols.

In [1]:
def nullable_symbols(grammar) -> set:
  nullable = set()
  prev_count = None

  def all_nullable_symbols(rule: str) -> bool:
    return all(map(lambda s: s in nullable or s == (0, ''), rule))

  while len(nullable) != prev_count:
    prev_count = len(nullable)
    nullable_symbols = [var for var, definition in grammar['vars'].items() 
                        if list(filter(all_nullable_symbols, definition)) != []]
    nullable = nullable.union(set(nullable_symbols))

  return nullable  

### **Definition**

$ \forall A \in N: FIRST(A) = a \in 𝛴 ∣ A ↠ aα$. 

If $A$ is the nullable nonterminal symbol, then the end of input $⊳$ is added to $FIRST(A)$.


In [2]:
def first(grammar, nonterminal: str) -> set:
  toks = set()
  if nonterminal in nullable_symbols(grammar):
    toks.add((0, ''))

  for rule in grammar['vars'][nonterminal]:
    if rule[0] in grammar['toks']:
      toks.add(rule[0])
    else:   
      toks.update(first(grammar, rule[0]))
    
  return toks

### **Definition**

$ \forall A \in N: FOLLOW(A) = a \in 𝛴 ∣ S↠𝛼Aa𝛽$. 

If $S↠𝛼A$, then the end of input $⊳$ is added to $FOLLOW(A)$.

In [10]:
def follow(grammar, nonterminal: str) -> set:
  toks = set()

  for s, rules in grammar['vars'].items():
    for rule in rules:
      indexes = [i for i,x in enumerate(rule) if (x == nonterminal and i != len(rule))]
      for i in indexes:
        if rule[i+1] in grammar['toks']:
          toks.add(rule[i+1])
        else:
          toks.update(first(grammar, rule[i+1]))

  return toks

In [12]:
def test():
  """
    S  -> TS'
    S' -> +TS' ∣ e
    T  -> FT'
    T' -> *FT' ∣ e
    F  -> v ∣ n ∣ (S)

    FIRST(S) = FIRST(T) = FIRST(F) = {v,n,(}
    FIRST(S') = {+,⊳}
    FIRST(T') = {*,⊳}

    FOLLOW(S) = FOLLOW(S') = {),⊳}
    FOLLOW(T) = FOLLOW(T') = {+,⊳}
    FOLLOW(F) = {*,+,⊳}
  """
  grammar = {
      'toks': {(0, ''), (1, '+'), (2, '*'), (3, 'v'), (4, 'n'), (5, '('), (6, ')')},
      'vars': {
          'S': [['T', "S'"]], 
          "S'": [[(1, '+'), 'T', "S'"], 
                 [(0, '')]], 
          'T': [['F', "T'"]],
          "T'": [[(2, '*'), 'F', "T'"],
                 [(0, '')]],
          'F': [[(3, 'v')],
                [(4, 'n')],
                [(5, '('), 'S', (6, ')')]]
          },
      'hvar': 'S'
      }

  assert(first(grammar, 'S') == {(3, 'v'), (4, 'n'), (5, '(')})
  assert(first(grammar, 'T') == first(grammar, 'S'))
  assert(first(grammar, 'F') == first(grammar, 'S'))
  assert(first(grammar, "S'") == {(1, '+'), (0, '')})
  assert(first(grammar, "T'") == {(2, '*'), (0, '')})

  print(follow(grammar, 'S'))
  assert(follow(grammar, 'S') == {(6, ')'), (0, '')})
  assert(follow(grammar, 'S') == follow(grammar, "S'"))
  assert(follow(grammar, 'T') ==  {(1, '+'), (0, '')})
  assert(follow(grammar, 'T') == follow(grammar, "T'"))
  assert(follow(grammar, "F") == {(1, '+'), (2, '*'), (0, '')})

test()  

{(6, ')')}


AssertionError: ignored